In [1]:
from dolfinx import io
from pathlib import Path
from mpi4py import MPI
from petsc4py.PETSc import ScalarType  # type: ignore
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, grad, inner

In [2]:
msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (2.0, 2.0)), n=(32, 16),
                            cell_type=mesh.CellType.quadrilateral)

In [3]:
V = fem.functionspace(msh, ("Lagrange", 1)) # quadratic polynomials

In [4]:
facets = mesh.locate_entities_boundary(msh, dim=(msh.topology.dim - 1),
                                       marker=lambda x: np.logical_or(np.isclose(x[0], 0.0),
                                                                      np.isclose(x[0], 2.0)))

In [5]:
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)  # DOF in the model

In [6]:
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [7]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = ufl.sin(5 * x[0])
a = inner(grad(u), grad(v)) * dx
L = inner(f, v) * dx + inner(g, v) * ds

In [8]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [9]:
with io.VTXWriter(MPI.COMM_WORLD, "uh_print.bp4", [uh], engine="BP4") as desplz: # aca realizamos impresion en VTK
    desplz.write(0.0)
    